In [ ]:
# PLAN
  # DEFINE ADDRESS ARRAY
  
  # DEFINE FILENAME QUEUE
  
  # DEFINE IMAGE READER
  
  # EXECUTE SESSION
    # create queue runner object
    
    # convert images to list of tensors
      # create empty list
      # read one file from the queue
      # decode
      # resize
      # set shape
      # sess.run to convert to Numpy array
      # add new dimension to array
      # append array to image list
    
    # stop processing
    
    # join threads
    
    # write image summary
    
  # VIEW IN TENSORBOARD
  

In [1]:
# IMPORT LIBRARIES
import tensorflow as tf
import google.datalab.ml as ml

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
# DEFINE ADDRESS ARRAY CONTAINING IMAGE ADDRESSES
original_image_list = ["./images/cat.2.jpg", 
              "./images/cat.3.jpg",
              "./images/cat.4.jpg",
              "./images/cat.5.jpg",
              "./images/cat.6.jpg",
              "./images/dog.1.jpg",
              "./images/dog.2.jpg",
              "./images/dog.3.jpg",
              "./images/dog.4.jpg",
              "./images/dog.5.jpg"
             ]

In [7]:
# DEFINE FILENAME QUEUE USING ADDRESS ARRAY
filename_queue = tf.train.string_input_producer(original_image_list) # we will produce multiple threads to read from this queue of images

In [8]:
# DEFINE IMAGE READER
image_reader = tf.WholeFileReader() # allows us to read the entire file in one go

In [9]:
# EXECUTE SESSION
with tf.Session() as sess:
  
  # create queue runner object to coordinate the loading of image files
  
  coord = tf.train.Coordinator() # class which allows you to coordinate multiple threads; a method call on the coordinator can be used to stop processing on all your threads
  
  threads = tf.train.start_queue_runners(sess = sess, coord = coord) # queue runner allows you to process elements in a queue in parallel; purpose of threads is to read from the queue of images; each image will be processed on a different thread
  
  
  
  # convert images to list of tensors
  
  image_list = []
  
  for i in range(len(original_image_list)):
    
    # read one file from the queue (using the image reader you created)
    _, image_file = image_reader.read(filename_queue) # image_reader.read returns a tuple (filename, filecontent); we will ignore the filename
    
    
    # decode
    image = tf.image.decode_jpeg(image_file)
    
    
    # resize
    image = tf.image.resize_images(image, [224,224])
    
    
    # set shape
    image.set_shape((224,224,3))
    
    
    # sess.run to convert to Numpy array
    image_array = sess.run(image)
    print image_array.shape
    
    
    # add new dimension to array, append array to image list
    image_list.append(tf.expand_dims(image_array, 0))
    

    
  # stop processing
  coord.request_stop()


  # join threads
  coord.join(threads)


  # write image summary
  summary_writer = tf.summary.FileWriter('./resizedimages', graph=sess.graph)
  
  index = 0
  
  for item in image_list:
    summary_str = sess.run(tf.summary.image("image-" + str(index), item)) # create a summary of your images
    summary_writer.add_summary(summary_str)
    index += 1
    
  summary_writer.close()
  
  

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)


In [10]:
# VIEW IN TENSORBOARD
tboard_001 = ml.TensorBoard.start('./resizedimages')

TensorBoard was started successfully with pid 3632. Click here to access it.

In [11]:
ml.TensorBoard.stop(tboard_001)